# Image Detection Demo with PytorchWildlife


This tutorial guides you on how to use PyTorchWildlife to separate positive and negative animal detections. We will go through the process of setting up the environment, defining the detection model, as well as performing inference and saving the results in different ways.

## Prerequisites

Install PytorchWildlife running the following commands:

```bash
conda create -n pytorch_wildlife python=3.8 -y
conda activate pytorch_wildlife
pip install PytorchWildlife
```

Also, make sure you have a CUDA-capable GPU if you intend to run the model on a GPU. This notebook can also run on CPU.

## Importing libraries

First, we'll start by importing the necessary libraries and modules.


In [2]:
import os
import torch
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife import utils as pw_utils

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/tobylaw/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Model Initialization

We will initialize the MegaDetectorV5 model for image detection. This model is designed for detecting animals in images.


In [3]:
# Setting the device to use for computations ('cuda' indicates GPU)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda":
    torch.cuda.set_device(0)

# Initializing the MegaDetectorV6 model for image detection
detection_model = pw_detection.MegaDetectorV6(
    device=DEVICE, pretrained=True, version="yolov9c"
)

# Uncomment the following line to use MegaDetectorV5 instead of MegaDetectorV6
# detection_model = pw_detection.MegaDetectorV5(device=DEVICE, pretrained=True, version="a")


Ultralytics 8.3.55 🚀 Python-3.9.6 torch-2.5.1 CPU (Apple M1)
YOLOv9c summary (fused): 384 layers, 25,321,561 parameters, 0 gradients, 102.3 GFLOPs


## Variable definition

In order to process the batch detection, we will define an input directory where the images are stored, a confidence threshold and an output directory to copy the positive and negative images into distinctive folders. If you want to follow this tutorial with your own data, modify the following variables.


In [19]:
# tgt_folder_path = "/Users/tobylaw/Documents/cape-liptrap"
tgt_folder_path = "./imgs"
output_path = "output"
classified_path = "classified"
threshold = 0.2

## Batch Image Detection

Next, we'll demonstrate how to process multiple images in batches. This is useful when you have a large number of images and want to process them efficiently.


In [16]:
results = detection_model.batch_image_detection(tgt_folder_path, batch_size=16)

  0%|          | 0/1 [00:00<?, ?it/s]


0: 480x640 1 animal, 417.0ms
1: 480x640 1 animal, 417.0ms
2: 480x640 (no detections), 417.0ms
3: 480x640 (no detections), 417.0ms
4: 480x640 (no detections), 417.0ms
5: 480x640 1 animal, 417.0ms
6: 480x640 (no detections), 417.0ms
7: 480x640 1 animal, 417.0ms
8: 480x640 1 animal, 417.0ms
Speed: 4.2ms preprocess, 417.0ms inference, 2.6ms postprocess per image at shape (9, 3, 480, 640)


100%|██████████| 1/1 [00:05<00:00,  5.21s/it]


## Separate positive and negative detections

PytorchWildlife allows to copy the files from your original folder to a new directory containing the "Animal" and "No-animal" subdirectories. A detection is considered positive if the prediction confidence is higher than the threshold


In [ ]:
os.makedirs(output_path, exist_ok=True)
os.makedirs(classified_path, exist_ok=True)
json_file = os.path.join(output_path, "detection_results.json")
pw_utils.save_detection_json(
    results,
    json_file,
    categories=detection_model.CLASS_NAMES,
    exclude_category_ids=[],  # Category IDs can be found in the definition of each model.
    exclude_file_path=tgt_folder_path,
)

pw_utils.save_detection_images(results, output_path)

# Separate the positive and negative detections through file copying:
pw_utils.detection_folder_separation(json_file, output_path, classified_path, threshold)

'9 files were successfully separated'

### Copyright (c) Microsoft Corporation. All rights reserved.

### Licensed under the MIT License.
